**Importing required libraries**

In [39]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [11]:
True_news = pd.read_csv("True.csv")
Fake_news = pd.read_csv("Fake.csv")

True_news['label'] = 0
Fake_news['label'] = 1

ds1 = True_news[['text','label']]
ds2 = Fake_news[['text','label']]

ds = pd.concat([ds1,ds2])
ds.shape

(44898, 2)

In [15]:
#check for null values
ds.isnull().sum()

text     0
label    0
dtype: int64

In [18]:
ds['label'].value_counts()

label
1    23481
0    21417
Name: count, dtype: int64

**Shuffling the data**

In [21]:
ds = ds.sample(frac =1)
ds

,text,label
7428,PRAGUE (Reuters) - Czech President Milos Zeman...,0
20070,JOHANNESBURG (Reuters) - A violent feud betwee...,0
6355,The mere fact that Donald Trump is the presump...,1
861,Chicago Police Sergeant George Granias abuse ...,1
18681,KABUL (Reuters) - Islamic State claimed respon...,0
...,...,...
619,CHICAGO/NEW YORK (Reuters) - State and local g...,0
21345,BANGKOK (Reuters) - Ousted Thai prime minister...,0
14789,Jeb takes a swipe at Trump and Trump hammers h...,1
12215,GENEVA (Reuters) - The United Nations human ri...,0


In [12]:
import nltk
import re
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adhir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adhir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Cleaning the text**

In [22]:
def clean_row(row):
    row = row.lower()
    row = re.sub('[^a-zA-Z]', ' ', row) 
    tokens = row.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words] 
    cleaned_text = ' '.join(tokens) 
    return cleaned_text

In [19]:
print(type(ds))
print(ds.columns)

<class 'pandas.core.frame.DataFrame'>
Index(['text', 'label'], dtype='object')


In [23]:
dataset['text']=dataset['text'].apply(lambda x : clean_row(x))

**Text Vectorzing**

In [26]:
vectorize=TfidfVectorizer(max_features = 5000 ,lowercase=False,ngram_range=(1,2))

In [27]:
x=dataset.iloc[:30000,0]
y=dataset.iloc[:30000,1]

In [30]:
train_data,test_data,train_label,test_label=train_test_split(x,y,test_size=0.2,random_state=0)

In [31]:
vec_train_data=vectorize.fit_transform(train_data)
vec_train_data=vec_train_data.toarray()

In [32]:
vec_test_data=vectorize.fit_transform(test_data)
vec_test_data=vec_test_data.toarray()

In [33]:
training_data=pd.DataFrame(vec_train_data,columns= vectorize.get_feature_names_out())
testing_data=pd.DataFrame(vec_test_data,columns= vectorize.get_feature_names_out())

**Creating the model**

In [36]:
clf=MultinomialNB()
clf.fit(training_data,train_label)

MultinomialNB()

In [37]:
y_pred=clf.predict(testing_data)
#accuracy_score()

In [40]:
y_pred_train=clf.predict(training_data)
accuracy_score(train_label,y_pred_train)

0.9465416666666666

In [43]:
txt = 'Coca-Cola to be banned in all countries starting 2025.'
news=clean_row(txt)
pred=clf.predict(vectorize.transform([news]).toarray())
if pred==0:
    print('True news')
else:
    print('Fake news')

Fake news


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
